In [ ]:
ДОМАШНЯЯ работа № 2

--схема БД: https://docs.google.com/document/d/1NVORWgdwlKepKq_b8SPRaSpraltxoMg2SIusTEN6mEQ/edit?usp=sharing

--task1
--Корабли: Для каждого класса определите число кораблей этого класса, потопленных в сражениях. Вывести: класс и число потопленных кораблей.

--task2
--Корабли: Для каждого класса определите год, когда был спущен на воду первый корабль этого класса. Если год спуска на воду головного корабля неизвестен, определите минимальный год спуска на воду кораблей этого класса. Вывести: класс, год.


--task5
--Компьютерная фирма: Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker


In [130]:
#проверка подключения PostgreSQL 
import psycopg2
import pandas as pd

DB_HOST = '52.157.159.24'

#####################
#!введите свои реквизиты!

DB_USER = 'student15'
DB_USER_PASSWORD = 'student15_password'
DB_NAME = 'sql_ex'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

In [106]:
#task1
#Корабли: Для каждого класса определите число кораблей этого класса, потопленных в сражениях. 
#Вывести: класс и число потопленных кораблей.

request = """

select c.class, count(result) from classes c  left join ships s 
on c.class=s.class left join Outcomes o on (s.name=o.ship or c.class= o.ship) where result='sunk' and s.class!=s.name
group by c.class

"""
pd.read_sql_query(request, conn)

,class,count
0,Kongo,1


In [108]:
# число кораблей по каждому классу
request = """
select class, count(class) from
(
select class, name from ships s where s.class!=s.name 
union
select class, class from classes 
)d
group by class
"""
pd.read_sql_query(request, conn)

,class,count
0,Tennessee,2
1,Revenge,4
2,Yamato,2
3,Kongo,4
4,Renown,3
5,North Carolina,3
6,Iowa,4
7,Bismarck,1


In [3]:
#task2
#Корабли: Для каждого класса определите год, когда был спущен на воду первый корабль этого класса. 
#Если год спуска на воду головного корабля неизвестен, определите минимальный год спуска на воду кораблей этого класса. 
# Вывести: класс, год.
request = """

select class, min(launched) from Ships group by class
UNION
select Ship, NULL from Outcomes 
where ship in (select class from Classes)
  and 
 ship not IN (select name from Ships)

"""
pd.read_sql_query(request, conn)

,class,min
0,Revenge,1916.0
1,Bismarck,NaN
2,Iowa,1943.0
3,Renown,1916.0
4,Tennessee,1920.0
5,Yamato,1941.0
6,Kongo,1913.0
7,North Carolina,1941.0


In [122]:
#task3
#-Корабли: Для классов, имеющих потери в виде потопленных кораблей и не менее 3 кораблей в базе данных, 
#вывести имя класса и число потопленных кораблей.

request = """

SELECT class, COUNT(result) AS sunk 
FROM 
(SELECT class, result, name 
 FROM Ships left join 
 Outcomes ON ship=name AND  result = 'sunk'
 union
 select class,result,ship from Classes c left join Outcomes o on class= ship AND  result = 'sunk'
 ) T 
GROUP BY class 
HAVING COUNT(class) >=3  and count(result)>0
 

"""
pd.read_sql_query(request, conn)

,class,sunk
0,Kongo,1


In [134]:
#-task4
#-Корабли: Найдите названия кораблей, имеющих наибольшее число орудий среди всех кораблей такого 
# же водоизмещения (учесть корабли из таблицы Outcomes).
request = """

select name from ships
where class in
  (select class from claSSES
         where displacement in (select displacement from CLASSES group by displacement having count(*) > 1) and 
          numguns in (select max(numguns) from CLASSES group by displacement having count(*) > 1 )
   )      
union
select ship from Outcomes 
where ship in
  (select class from claSSES
         where displacement in (select displacement from CLASSES group by displacement having count(*) > 1) and 
          numguns in (select max(numguns) from CLASSES group by displacement having count(*) > 1 )
   )  
         
"""
pd.read_sql_query(request, conn)

,name
0,Tennessee
1,California


In [ ]:
#--task5
#--Компьютерная фирма:
# Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM 
# и с самым быстрым процессором среди всех ПК, 
# имеющих наименьший объем RAM. Вывести: Maker

request = """
SELECT DISTINCT maker FROM Product 
WHERE type = 'printer' 
AND 
      maker IN (select maker from Product 
where model in (select model from pc 
where ram in (select min(ram) from pc)
and (speed in (select max(speed) from pc 
where ram in  (select min(ram) from pc))
               )))
group by maker;
"""
pd.read_sql_query(request, conn)